In [1]:
import pandas as pd
import numpy as np
import sys, os
if not os.path.exists('OSAT') and os.path.exists('../OSAT'):
    sys.path.insert(1, os.path.abspath('..'))


from OSAT import thermometers
from OSAT.tools import vec_mc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_ol = pd.read_excel("input_ol.xlsx")
df_sp = pd.read_excel("input_sp.xlsx")
df_ol_err =  pd.read_excel("input_ol_err.xlsx")
df_sp_err = pd.read_excel("input_sp_err.xlsx")

In [3]:
mc_col_sp = df_sp_err.columns[1::]
mc_col_sp

Index(['SiO2', 'TiO2', 'Al2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'NiO',
       'ZnO'],
      dtype='object')

In [4]:
mc_col_ol = df_ol_err.columns[1:-1]
mc_col_ol

Index(['SiO2', 'TiO2', 'Al2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'P2O5',
       'Cr2O3'],
      dtype='object')

In [5]:
mc = 1000

In [6]:
sp_mc_collect = pd.DataFrame()
for idx in range(len(df_sp)):
    df_iter = vec_mc(df_sp[mc_col_sp], df_sp_err[mc_col_sp], idx, mc)
    sp_mc_collect = pd.concat((sp_mc_collect, df_iter))

In [7]:
ol_mc_collect = pd.DataFrame()
for idx in range(len(df_ol)):
    df_iter = vec_mc(df_ol[mc_col_ol], df_ol_err[mc_col_ol], idx, mc)
    ol_mc_collect = pd.concat((ol_mc_collect, df_iter))

In [8]:
sp_mc_collect.columns = mc_col_sp
sp_mc_collect[sp_mc_collect < 0] = 0

In [9]:
ol_mc_collect.columns = mc_col_ol
ol_mc_collect[ol_mc_collect < 0] = 0

In [10]:
sp_mc_collect['sample_group'] = df_sp.loc[df_sp.index.repeat(mc)]['sample_no']  # avoid merge error in the script of alolivthermo
ol_mc_collect['sample_group'] = df_ol.loc[df_ol.index.repeat(mc)]['sample_no']

In [11]:
ol_mc_collect.reset_index(drop=True, inplace=True)
sp_mc_collect.reset_index(drop=True, inplace=True)

In [12]:
ol_mc_collect['sample_no'] = ol_mc_collect.index  # sampel_no match use
sp_mc_collect['sample_no'] = sp_mc_collect.index

In [13]:
reg = thermometers.models(ol_mc_collect, sp_mc_collect).compute()

/Users/easonzz/Documents/GitHub/olspthermo/OSAT/thermometers.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.15646937 0.17417633 0.16848681 0.17527978 0.16880729 0.16310789
 0.16963957 0.17990422 0.17972923 0.166713   0.17481105 0.17500221
 0.16446584 0.17630957 0.17387904 0.17022918 0.17843134 0.17893552
 0.17017658 0.17486761 0.16228458 0.16096112 0.15986468 0.17728526
 0.17419061 0.16572498 0.17481244 0.15988382 0.15941422 0.16530505
 0.17642359 0.16467565 0.16440062 0.17711505 0.17066227 0.17601433
 0.16069671 0.17012473 0.16974582 0.1727955  0.17071311 0.17712556
 0.16670195 0.17675783 0.17150905 0.15960315 0.16580598 0.17697194
 0.16044814 0.17588895 0.17355768 0.16531894 0.16986906 0.16797097
 0.17108111 0.16858007 0.17625416 0.17715388 0.18696522 0.17223532
 0.16648411 0.16396808 0.16336979 0.17320144 0.16037198 0.16275096
 0.17003531 0.16725526 0.16728277 0.17915107 0.17113227 0.1707891
 0.17630358

In [14]:
reg.head()

,SiO2_ol,TiO2_ol,Al2O3_ol,FeO_ol,MnO_ol,MgO_ol,CaO_ol,Na2O_ol,P2O5_ol,Cr2O3_ol,sample_group_ol,sample_no,SiO2,TiO2,Al2O3,Cr2O3,FeO,MnO,MgO,CaO,NiO,ZnO,sample_group,SiO2_xca,TiO2_xca,Al2O3_xca,Cr2O3_xca,FeO_xca,MnO_xca,MgO_xca,sumcat_xca,sumchg_xca,fe3_xca,fe2_xca,fTet_xca,fOct_xca,proj_xca,mFeCr2O4_xca,mFe/MnAl2O4_xca,mFe3O4_xca,mMgAl2O4_xca,mFe2TiO4_xca,sum_comps_xca,X2_xca,X3_xca,X4_xca,X5_xca,reg_X2(1+X4-X2),reg_(1-X2)X3,reg_(1-X2)X4,reg_(1-X2)X5,reg_X3(X3+X4+X5),reg_X4(X3+X4+X5),reg_X5(X3+X4+X5),reg_X3X4,reg_X3X5,reg_X4X5,reg_X2pow0.5,reg_X2pow2,reg_X2,Cr#,lnkdal,lnkdcr,Fo,t_thermo,t_kdcr,t_kdal,t_coogan,t_zThermoAlCr,err_zThermoAlCr,t_zThermoAl,err_zThermoAl,t_zThermoCr,err_zThermoCr,z_thermo_cr,z_thermo_al,p_thermo_cr,p_thermo_al
0,39.914470,0.018932,0.033965,17.433142,0.268311,42.183066,0.248900,0.0,0.0,0.048757,PI082_02,0,0.072159,2.329693,25.534309,25.796584,30.541449,0.254299,11.467023,0.005937,0.0,0.0,PI082_02,0.001201,0.029170,0.500863,0.339450,0.425105,0.003585,0.284510,1.582684,3.213810,0.156469,0.268636,1,1,1.0,0.169725,-0.034079,0.078235,0.284510,0.029170,0.527561,0.539294,0.321716,0.055292,0.148295,0.237724,0.148217,0.025474,0.068320,0.168999,0.029045,0.077900,0.017788,0.047709,0.008200,0.734366,0.290838,0.539294,0.403954,-6.622450,-6.271148,0.811790,1087.228825,1143.828874,1169.307909,1181.938682,1169.307909,43.3,1169.307909,43.3,1143.828874,34.2,1.356551,1.659569,0.087462,0.048501
1,40.117551,0.019154,0.033346,16.973472,0.265218,42.434449,0.238170,0.0,0.0,0.007150,PI082_02,1,0.072352,2.375931,25.151599,25.687625,32.312367,0.256592,11.409238,0.005319,0.0,0.0,PI082_02,0.001204,0.029749,0.493356,0.338016,0.449755,0.003617,0.283077,1.597570,3.186501,0.174176,0.275579,1,1,1.0,0.169008,-0.036399,0.087088,0.283077,0.029749,0.532523,0.531576,0.317372,0.055864,0.163539,0.245589,0.148665,0.026168,0.076605,0.170358,0.029987,0.087784,0.017730,0.051903,0.009136,0.729093,0.282573,0.531576,0.406574,-6.625725,-8.186615,0.816730,1088.868107,1008.048349,1168.095818,1180.827271,1008.048349,34.2,1168.095818,43.3,1088.868107,23.9,1.937032,1.601917,0.026371,0.054587
2,40.131328,0.020729,0.036746,15.900267,0.264498,42.612786,0.242353,0.0,0.0,0.045887,PI082_02,2,0.073957,2.284161,25.083314,26.072755,31.570233,0.253682,11.465768,0.005762,0.0,0.0,PI082_02,0.001231,0.028600,0.492016,0.343084,0.439425,0.003576,0.284479,1.591181,3.195813,0.168487,0.270938,1,1,1.0,0.171542,-0.038471,0.084243,0.284479,0.028600,0.530394,0.536355,0.323424,0.053922,0.158832,0.239967,0.149954,0.025001,0.073642,0.173413,0.028912,0.085162,0.017440,0.051370,0.008565,0.732363,0.287677,0.536355,0.410827,-6.525938,-6.342455,0.826906,1105.295270,1150.280741,1185.224103,1199.183849,1185.224103,43.3,1185.224103,43.3,1105.295270,23.9,1.078181,1.616093,0.140477,0.053037
3,40.012682,0.021343,0.035767,16.466338,0.268173,42.438489,0.240959,0.0,0.0,0.034214,PI082_02,3,0.070598,2.268128,25.380354,24.758371,31.929955,0.253778,11.427282,0.005469,0.0,0.0,PI082_02,0.001175,0.028399,0.497843,0.325789,0.444432,0.003577,0.283524,1.583565,3.158696,0.175280,0.269152,1,1,1.0,0.162894,-0.034603,0.087640,0.283524,0.028399,0.527855,0.537126,0.308597,0.053801,0.166030,0.239156,0.142842,0.024903,0.076851,0.163071,0.028430,0.087735,0.016603,0.051236,0.008933,0.732889,0.288504,0.537126,0.395549,-6.564701,-6.584280,0.821240,1106.310631,1128.109413,1181.814182,1194.588587,1181.814182,43.3,1181.814182,43.3,1106.310631,23.9,0.522458,1.526618,0.300676,0.063428
4,40.101464,0.019483,0.034690,16.710279,0.271754,42.339703,0.239133,0.0,0.0,0.039697,PI082_02,4,0.080914,2.241288,25.317158,25.312545,31.386352,0.252668,11.436450,0.004011,0.0,0.0,PI082_02,0.001347,0.028063,0.496603,0.333081,0.436866,0.003562,0.283752,1.581927,3.175933,0.168807,0.268058,1,1,1.0,0.166540,-0.035450,0.084404,0.283752,0.028063,0.527309,0.538113,0.315831,0.053220,0.160065,0.237921,0.145878,0.024581,0.073932,0.167111,0.028159,0.084693,0.016808,0.050553,0.008519,0.733562,0.289566,0.538113,0.401453,-6.592795,-6.457790,0.818726,1099

In [15]:
reg.groupby('sample_group')[['t_thermo', 't_kdcr', 't_kdal', 't_coogan','t_zThermoAlCr', 't_zThermoAl', 't_zThermoCr']].agg(['mean', 'std'])

t_thermo                 t_kdcr                  t_kdal  \
                     mean       std         mean        std         mean   
sample_group                                                               
PI082_02      1090.783722  9.987154  1115.888642  67.973201  1171.152923   

                          t_coogan           t_zThermoAlCr             \
                  std         mean       std          mean        std   
sample_group                                                            
PI082_02      8.93198  1183.799032  9.361181    1166.70004  66.665608   

              t_zThermoAl             t_zThermoCr             
                     mean        std         mean        std  
sample_group                                                  
PI082_02      1170.765863  10.754714  1098.683806  21.512562